* http://nbviewer.ipython.org/github/vals/scilife-python-course/blob/master/parallel%20python.ipynb

In [1]:
import IPython.parallel

Run ```ipcluster start -n 10``` to start the cluster

Connect from notebook to cluster

In [2]:
clients = IPython.parallel.Client(profile='default')

In [3]:
clients.ids

[0, 1, 2, 3]

In [4]:
dview = clients[:]

# Running code

In [5]:
%%px
import os
import numpy as np

In [7]:
%%px
print(os.getpid())

[stdout:0] 33387
[stdout:1] 33388
[stdout:2] 33389
[stdout:3] 33390


In [9]:
%%px --targets 1,2
print(os.getpid())

[stdout:1] 33388
[stdout:2] 33389


In [10]:
%%px
data = np.random.rand(10)

In [27]:
def square_this(x):
    return x**2

In [39]:
values = np.arange(10)

In [40]:
dview = clients[:]

In [43]:
dview.map_sync(square_this, values)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [35]:
r = v.apply_async(square_this, 4)

In [38]:
r.progress

2

# Running functions

In [11]:
def get_pid(x):
    return 'pid=%d: %d' % (os.getpid(), x)

In [12]:
dview.apply_sync(get_pid, 10)

['pid=33387: 10', 'pid=33388: 10', 'pid=33389: 10', 'pid=33390: 10']

In [13]:
def compute_square(x):
    return str(x**2)

In [14]:
dview.apply_sync(compute_square, np.arange(10))

['[ 0  1  4  9 16 25 36 49 64 81]',
 '[ 0  1  4  9 16 25 36 49 64 81]',
 '[ 0  1  4  9 16 25 36 49 64 81]',
 '[ 0  1  4  9 16 25 36 49 64 81]']

In [15]:
dview.map_sync(compute_square, np.arange(10))

['0', '1', '4', '9', '16', '25', '36', '49', '64', '81']

In [16]:
@dview.parallel(block=True)
def psquare(x):
    return str(x**2)

In [17]:
psquare(np.arange(20)) # partition args; run fun on partitions

['[ 0  1  4  9 16]',
 '[25 36 49 64 81]',
 '[100 121 144 169 196]',
 '[225 256 289 324 361]']

In [18]:
psquare.map(np.arange(5)) # apply fun on each argument

['0', '1', '4', '9', '16']

# Distributing data

In [19]:
dview['a'] = np.arange(10)

In [20]:
dview['a']

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]

In [21]:
dview.push({'a': np.arange(10, 15)}, targets=[0, 1], block=True)

[None, None]

In [22]:
t = dview.pull('a', targets=[0, 1], block=False)
t.get()

[array([10, 11, 12, 13, 14]), array([10, 11, 12, 13, 14])]

In [23]:
dview['a']

[array([10, 11, 12, 13, 14]),
 array([10, 11, 12, 13, 14]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]

In [24]:
dview.scatter('todo', np.arange(20))

<AsyncResult: scatter>

In [25]:
dview['todo']

[array([0, 1, 2, 3, 4]),
 array([5, 6, 7, 8, 9]),
 array([10, 11, 12, 13, 14]),
 array([15, 16, 17, 18, 19])]

# Connecting to remove controler

In [18]:
client = IPython.parallel.Client(profile_dir='/Users/angermue/mnt/ebi/.ipython/profile_mp', sshserver='Ebii', password='FR65masEI11b')

In [20]:
client = IPython.parallel.Client(url_file='/Users/angermue/mnt/ebi/.ipython/profile_mp/security/ipcontroller-client.json', sshserver='Ebii', password='FR65masEI11b')

In [21]:
len(client)

5

In [7]:
client.close()